#  Loading the Dataset & Preparing the Data

In [1]:
import tensorflow as tf
import numpy as np

# Load the dataset (Shakespeare Sonnets)
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

# Read the text
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print(f'Length of text: {len(text)} characters')

# Get unique characters in the dataset
vocab = sorted(set(text))
vocab_size = len(vocab)
print(f'{vocab_size} unique characters')

# Create a mapping from characters to numbers
char2idx = {u: i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

# Convert text to integers (character indices)
text_as_int = np.array([char2idx[c] for c in text])

# Maximum length of sequences for training
seq_length = 100
examples_per_epoch = len(text) // (seq_length + 1)

# Create training examples
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

# Create sequences of the desired length
sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

# Split input and target
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

# Creating training batches
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)


1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Length of text: 1115394 characters
65 unique characters


# Define & Train the Model

In [2]:
# Define the model
embedding_dim = 256
rnn_units = 1024

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim),
    tf.keras.layers.LSTM(rnn_units,
                         return_sequences=True,
                         stateful=True,
                         recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
])

# Compile the model
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

# Train the model
EPOCHS = 10
history = model.fit(dataset, epochs=EPOCHS)


Epoch 1/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 17s 65ms/step - loss: 2.8849
Epoch 2/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 13s 65ms/step - loss: 1.8657
Epoch 3/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 13s 66ms/step - loss: 1.6046
Epoch 4/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 22s 67ms/step - loss: 1.4803
Epoch 5/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 13s 68ms/step - loss: 1.4048
Epoch 6/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 14s 69ms/step - loss: 1.3530
Epoch 7/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 14s 69ms/step - loss: 1.3064
Epoch 8/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 21s 70ms/step - loss: 1.2733
Epoch 9/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 14s 71ms/step - loss: 1.2381
Epoch 10/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 14s 71ms/step - loss: 1.2069


In [3]:
# Save the trained model weights
model.save_weights('text_generator.weights.h5')


# Reload the Model for Text Generation

In [4]:
import tensorflow as tf

# Recreate the model architecture for inference
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim),
    tf.keras.layers.LSTM(rnn_units,
                         return_sequences=True,
                         stateful=True,
                         recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
])

# Build the model with batch size = 1 for inference
model.build(tf.TensorShape([1, None]))  # This is where the input shape is defined properly

# Load the saved weights
model.load_weights('text_generator.weights.h5')


# Define the Text Generation Function


In [5]:
def generate_text(model, start_string, temperature=1.0):
    # Number of characters to generate
    num_generate = 1000

    # Convert start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store results
    text_generated = []

    # Reset states for stateful LSTM (Accessing LSTM layer directly)
    model.layers[1].reset_states()  # Resetting only the LSTM layer, which is at index 1

    for _ in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)

        # Apply temperature
        predictions = predictions / temperature

        # Sample from the distribution
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Ensure the predicted_id is within range
        if predicted_id >= vocab_size:
            predicted_id = np.random.randint(0, vocab_size)

        # Pass the predicted character as the next input to the model
        input_eval = tf.expand_dims([predicted_id], 0)

        # Append the predicted character to the generated text
        text_generated.append(idx2char[predicted_id])

    return start_string + ''.join(text_generated)


#  Generating Text

In [6]:
generated_text = generate_text(model, start_string="ROMEO: ", temperature=0.5)
print(generated_text)

ROMEO: stand to her so far for the sea,
As she was nothing in the best of the presence and the back way
To the actions of the maid of England;
And hanging them not the shadow of his face,
And that you shall not stay away the ground
From the power of the shadow of your father,
Richard the recompense of the world,
And give him thanks for him; which was the heart of me
That bare me still be too much. Well, I'll go to thee,
That stays here being so much a cousin
That I have fair words, and the poor devil's dear days
Had natural marriage which you have
sent for this prophecian that makes a thing,
While I did well be the fire of heaven,
That live shall not wear the state of the heart, and the great sorrow
And bring my father, and the storms did
Than the precious crown is straited
Which never been drinks it off,
And that the world is scarce the field to seek him that
But that I shall be past to my dear lord;
But let the streets to the king and stands and prove
As mine eyes me with me a more t